# Get Data
Daten von der BGG-API abfragen und in einen Dataframe speichern.

In [39]:
import requests
import xml.etree.ElementTree as ET
import json
import xmltodict
import pandas as pd

# Aktuelle Hitliste laden
Hier wird die auf BGG bereitgestellte csv-Datei verwendet

In [40]:
games = pd.read_csv("../data/boardgames_ranks.csv")

In [41]:
ngames = 500
games = games.iloc[:ngames,:].copy()

In [42]:
ids = games["id"]

# API anfragen

In [43]:
def bggApiFromID(id):
    
    response = requests.get(f"https://api.geekdo.com/xmlapi/boardgame/{id}?")
    
    root = ET.fromstring(response.content)
    
    result = {}
    for boardgame in root.findall('.//boardgame'):
        for element in boardgame:
            if element.tag != "poll":
                #wenn das bereits in result, dann anfügen, sonst neu anlegen 
                if element.tag in result:
                    result[element.tag].append(element.text)
                else:
                    result[element.tag] = [element.text]

            else:
                result[element.get('name')] = []
                for child in element:
                    for subchild in child:
                        result[element.get('name')].append(subchild.attrib)

    #wenn nur ein Element drin, dann kein Array mehr draus machen
    for key in result:
        if len(result[key])==1:
            result[key] = result[key][0]

    resultdf = pd.DataFrame()
    for key in result:
        resultdf[key] = ""
        resultdf.loc[0,key] = str(result[key])

    return resultdf

In [44]:
# die ersten x ranks abfragen und in DF speichern
bgdata = pd.DataFrame()
for i in range(ngames):
    bgdata = bgdata.append(bggApiFromID(ids[i]))

In [45]:
bgdata

,yearpublished,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,age,name,description,thumbnail,...,boardgameintegration,boardgamegraphicdesigner,boardgameeditor,boardgamewriter,boardgamesculptor,boardgamesolodesigner,boardgamecompilation,boardgameinsertdesigner,boardgame,boardgameissuearticle
0,2018,2,4,120,60,120,14,"['Brass: Birmingham', 'Brass. Бирмингем', 'Bra...",Brass: Birmingham is an economic strategy game...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2015,2,4,60,60,60,13,"['Pandemic Legacy: Saison 1', 'Pandemic Legac...",Pandemic Legacy is a co-operative campaign gam...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2017,1,4,120,60,120,14,"['Gloomhaven', 'Gloomhaven, aventures à Havren...",Gloomhaven is a game of Euro-inspired tactica...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,...,"['Frosthaven', 'Gloomhaven: Jaws of the Lion',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021,1,4,150,90,150,14,"['Archa Nova', 'Arche Nova', 'Ark Nova', 'Нови...","In Ark Nova, you will plan and design a modern...",https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,...,NaN,"['Steffen Bieker', 'Christof Tisch']","['Frank Heeren', 'Inga Keutmann', 'Bastian Win...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2017,3,6,480,240,480,14,"['Twilight Imperium: Čtvrtá edice', 'Twilight ...",Twilight Imperium (Fourth Edition) is a game o...,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,...,NaN,Shaun Boyke,NaN,"['Adam Baker', 'Daniel Clark (I)']",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2019,2,5,60,45,60,10,"['Hadara', 'Хадара', 'ハダラ', '哈達拉', '하다라']",Hadara carries you off into the world of cultu...,https://cf.geekdo-images.com/17wJNXSBYuVmKryV9...,...,NaN,NaN,Julia Faeta,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2014,1,4,180,180,180,0,Fire in the Lake,(from GMT website:)<br/><br/>Volume IV in GMT&...,https://cf.geekdo-images.com/08V8nzzLAr190Miv9...,...,NaN,Rodger B. MacGowan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2018,2,4,15,5,15,7,"['Zombie Kidz Evolucija', 'Zombie Kidz Evolúci...",Zombie Kidz Evolution is the first &quot;legac...,https://cf.geekdo-images.com/Iwv0Vg8FXIKlEr8xK...,...,"['Zombie Teenz Evolution', 'Zombie Teenz Evolu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2016,2,8,60,30,60,10,"['Citadela', 'Citadela DeLuxe', 'Citadele', 'C...","In Citadels, players take on new roles each ro...",https://cf.geekdo-images.com/sHd0jkZZLDgixHjAX...,...,NaN,"['Jasmine Radue', 'Samuel R. Shimota']",NaN,NaN,Samuel R. Shimota,NaN,NaN,NaN,NaN,NaN


In [46]:
games.reset_index(inplace=True, drop=True)
bgdata.reset_index(inplace=True, drop=True)

In [47]:
final = pd.concat([games,bgdata],1)
final

,id,name,yearpublished,rank,bayesaverage,average,usersrated,abstracts_rank,cgs_rank,childrensgames_rank,...,boardgameintegration,boardgamegraphicdesigner,boardgameeditor,boardgamewriter,boardgamesculptor,boardgamesolodesigner,boardgamecompilation,boardgameinsertdesigner,boardgame,boardgameissuearticle
0,224517,Brass: Birmingham,2018,1,8.41720,8.60057,43723,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.38410,8.53050,52692,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174430,Gloomhaven,2017,3,8.36528,8.59903,61211,NaN,NaN,NaN,...,"['Frosthaven', 'Gloomhaven: Jaws of the Lion',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,342942,Ark Nova,2021,4,8.33138,8.53762,40218,NaN,NaN,NaN,...,NaN,"['Steffen Bieker', 'Christof Tisch']","['Frank Heeren', 'Inga Keutmann', 'Bastian Win...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24165,8.60615,22955,NaN,NaN,NaN,...,NaN,Shaun Boyke,NaN,"['Adam Baker', 'Daniel Clark (I)']",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,269144,Hadara,2019,496,6.95822,7.41027,6959,NaN,NaN,NaN,...,NaN,NaN,Julia Faeta,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,144189,Fire in the Lake,2014,497,6.95724,8.06010,2893,NaN,NaN,NaN,...,NaN,Rodger B. MacGowan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,256952,Zombie Kidz Evolution,2018,498,6.95565,7.63395,4916,NaN,NaN,1.0,...,"['Zombie Teenz Evolution', 'Zombie Teenz Evolu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,205398,Citadels,2016,499,6.95545,7.25539,13605,NaN,NaN,NaN,...,NaN,"['Jasmine Radue', 'Samuel R. Shimota']",NaN,NaN,Samuel R. Shimota,NaN,NaN,NaN,NaN,NaN


In [48]:
final.to_pickle("../data/bg_data20240302.pkl")